# Imports

In [ ]:
import pandas     as pd
import numpy      as np
import seaborn    as sns
import regex      as re
import umap.umap_ as umap

import inflection
from matplotlib              import pyplot        as    plt
from plotly                  import express       as    px

from sklearn                 import mixture       as    mx
from sklearn                 import cluster       as    c
from sklearn                 import metrics       as    m
from sklearn                 import manifold      as    mn
from sklearn                 import decomposition as    dd
from scipy.cluster           import hierarchy     as    hr
from sklearn.preprocessing   import MinMaxScaler
from sklearn.ensemble        import RandomForestRegressor 


# Helper funtions

In [ ]:
from IPython.core.display    import HTML
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    sns.set()
jupyter_settings()

# load data

In [ ]:
df_raw = pd.read_csv('C:/Users/Lavin/Documents/Comunidade DS/insiders clustering/data/raw/Ecommerce.csv',encoding= 'unicode_escape')

In [ ]:
df_raw.head()

In [ ]:
df1 = df_raw.copy()

**data dimensions**

In [ ]:
df1.shape

**Rename columns**

In [ ]:
old_cols = df1.columns
snakecase = lambda x: inflection.underscore(x)
new_cols = list(map(snakecase,old_cols))
df1.columns = new_cols

In [ ]:
df1.columns

**delete column**

In [ ]:
df1.drop(columns='unnamed: 8',inplace=True)

In [ ]:
df1.head()

# Data types

In [ ]:
df1.dtypes

# Checking NA

In [ ]:
df1.isnull().sum()/len(df_raw)*100

## replace NA

In [ ]:
df_missing = df1.loc[df1['customer_id'].isna(),:]
df_missing.head()

In [ ]:
df_not_missing = df1.loc[~df1['customer_id'].isna(),:]
df_not_missing

In [ ]:
# create reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000,19000+len(df_backup),1)

#merge original with reference
df1 = pd.merge(df1,df_backup,on='invoice_no',how='left')

#coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

df1 = df1.drop(columns=['customer_id_x','customer_id_y'])

In [ ]:
df1.isnull().sum()

In [ ]:
df_backup.head()

In [ ]:
1- df1.shape[0]/df_raw.shape[0]

In [ ]:
df1.isnull().sum()

# Changing types

In [ ]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'],format='%d-%b-%y')

In [ ]:
#df1['invoice_no'].astype(int)

In [ ]:
df1.dtypes

'Invoice_no' possui letras em algumas linhas, necessita de limpeza mais detalhada

# Data description

In [ ]:
df1.head()

'invoice_no', : categorical atributtes

'stock_code': categorical atributtes

'description': categorical atributtes

'quantity': numerical atributtes

'invoice_date': numerical atributtes

'unit_price': numerical atributtes

'customer_id': categorical atributtes

'country': categorical atributtes

Remover 'European Community' e 'Unspecified'

# Filter

In [ ]:
df2 = df1.copy()

In [ ]:
# unit price > 0.0
df2 = df2.loc[df2['unit_price'] > 0.040,:]

#stock code != ['POST','D','M','DOT','CRUK','C2']
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK','C2'])]

# description
df2.drop(columns='description',inplace=True)

# country
df2 = df2[~df2['country'].isin(['European Community','Unspecified'])]


df2 = df2[~df2['customer_id'].isin([16446])]


# description
## quantity - Negative numbers means product returns
df_returns = df2.loc[df2['quantity']<0,:]
df_purchase = df2.loc[df2['quantity']>0,:]

# Features enginering

In [ ]:
df3 = df2.copy()

In [ ]:
df3.head()

## Feature creation

In [ ]:
df_ref = pd.DataFrame(df3['customer_id'])

In [ ]:
df_ref =  df_ref.drop_duplicates(ignore_index=True)

### Receita bruta

**Gross Revenue**

In [ ]:
df_purchase.loc[:,'gross_revenue'] = df_purchase.loc[:,'quantity'] * df_purchase.loc[:,'unit_price']
df_monetary = df_purchase[['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref,df_monetary,on='customer_id',how='left')

### Rencencia - dias da ultima compra

In [ ]:
df_recency = df_purchase[['customer_id','invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max()- df_recency['invoice_date']).dt.days
df_recency = df_recency.drop(columns = 'invoice_date')
df_ref = pd.merge(df_ref,df_recency,on='customer_id',how='left')

### Quantidade de produtos comprados

In [ ]:
df_prod_quant = (df_purchase[['customer_id','stock_code']].groupby('customer_id')
                                                       .count()
                                                       .reset_index()
                                                       .rename(columns = {'stock_code':'qtde_products'}))
df_ref = pd.merge(df_ref,df_prod_quant,on='customer_id',how='left')

### Frequência

**Frequência**

In [ ]:
df_aux = (df3[['customer_id','invoice_no','invoice_date']].drop_duplicates()
                                                          .groupby('customer_id')
                                                          .agg(max_ = ('invoice_date','max'),
                                                               min_ = ('invoice_date','min'),
                                                               days_ = ('invoice_date',lambda x:(x.max()-x.min()).days+1),
                                                               buy_  = ('invoice_date','count'))
                                                          .reset_index())
                                                          

df_aux['freq'] = df_aux.apply(lambda x: x['buy_']/x['days_'] if x['days_'] !=0 else 0,axis=1)
df_ref = pd.merge(df_ref,df_aux[['customer_id','freq']],on='customer_id',how='left')

### Devoluções

**Returns**

In [ ]:
df_avg_returns = df_returns[['customer_id','quantity']].drop_duplicates().groupby('customer_id').sum().abs().reset_index().rename(columns={'quantity':'qtde_returns'})
df_ref = pd.merge(df_ref,df_avg_returns,on='customer_id',how='left')
df_ref.loc[df_ref['qtde_returns'].isna(),'qtde_returns'] = 0

In [ ]:
df_ref = df_ref.dropna()

In [ ]:
df4 = df_ref.copy()

In [ ]:
df4.shape

# EDA

In [ ]:
df5 = df4.copy()

In [ ]:
df_aux = df5.drop(columns='customer_id')

## Estudo do Espaço

In [ ]:
#cols_select = ['gross_revenue', 'recency_days', 'qtde_products', 'freq', 'qtde_returns']

In [ ]:
#df_aux = df_aux[cols_select]

In [ ]:
df_aux.head()

In [ ]:
mm = MinMaxScaler()

df_aux['gross_revenue'] = mm.fit_transform(df_aux[['gross_revenue']])
df_aux['recency_days'] = mm.fit_transform(df_aux[['recency_days']])
df_aux['qtde_products'] = mm.fit_transform(df_aux[['qtde_products']])
df_aux['freq'] = mm.fit_transform(df_aux[['freq']])
df_aux['qtde_returns'] = mm.fit_transform(df_aux[['qtde_returns']])

In [ ]:
X = df_aux.copy()

## PCA

In [ ]:
pca = dd.PCA( n_components=X.shape[1] )

principal_components = pca.fit_transform( X )

# plot explained variable
features = range( pca.n_components_ )

plt.bar( features, pca.explained_variance_ratio_, color='black' )

# pca component
df_pca = pd.DataFrame( principal_components )

In [ ]:
sns.scatterplot( x=0, y=1, data=df_pca );

## UMAP

In [ ]:
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( X )

# embedding
df_umap = pd.DataFrame()
df_umap['embedding_x'] = embedding[:, 0]
df_umap['embedding_y'] = embedding[:, 1]

# plot UMAP
sns.scatterplot( x='embedding_x', 
                 y='embedding_y', 
                 data=df_umap )

## T-SNE

In [ ]:
reducer = mn.TSNE( n_components=2, n_jobs=-1, random_state=42 )
embedding = reducer.fit_transform( X )

# embedding
df_tsne = pd.DataFrame()
df_tsne['embedding_x'] = embedding[:, 0]
df_tsne['embedding_y'] = embedding[:, 1]

# plot UMAP
sns.scatterplot( x='embedding_x', 
                 y='embedding_y', 
                 data=df_tsne )

### Tree-based embedding

In [ ]:
X = df_aux.drop(columns=['gross_revenue'],axis=1)
y = df_aux['gross_revenue']


# model definition
rf_model = RandomForestRegressor(n_estimators=100,random_state=42)

#model training
rf_model.fit(X,y.values)

# Leaf
df_leaf = pd.DataFrame(rf_model.apply(X))
#dataframe Leaf

In [ ]:
df_leaf.head()

In [ ]:
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)


#embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]


sns.scatterplot(x='embedding_x', y ='embedding_y',data=df_tree);

# Data preparation

Distribuição normal e não possui outlier - stardart scaler

distribuição normal e possui outilier - robust scaler

não distribuição normal - Min max Scaler


**Teste de normalidade**

**QQ plot** - Quantile Quantile Plot ( Quatile teorica x Quatile real)

**KS Teste** - Kolgomorov Smirnoff ( Teste de Hipótese ) se P-valor > 0.5 Distribuição normal, P-valor < 0.5 - Não distribuição normal

In [ ]:
# df6 = df5.copy()

In [ ]:
# df6.head()

In [ ]:
# mm = MinMaxScaler()
# #ss = StandardScaler()
# #rs = RobustScaler()

# df6['gross_revenue'] = mm.fit_transform(df6[['gross_revenue']])
# df6['recency_days'] = mm.fit_transform(df6[['recency_days']])
# df6['qtde_invoice'] = mm.fit_transform(df6[['qtde_invoice']])
# df6['qtde_items'] = mm.fit_transform(df6[['qtde_items']])
# df6['qtde_products'] = mm.fit_transform(df6[['qtde_products']])
# df6['avg_basket_size'] = mm.fit_transform(df6[['avg_basket_size']])
# df6['avg_unique_basket_size'] = mm.fit_transform(df6[['avg_unique_basket_size']])
# df6['freq'] = mm.fit_transform(df6[['freq']])
# df6['avg_rec_days'] = mm.fit_transform(df6[['avg_rec_days']])
# df6['qtde_returns'] = mm.fit_transform(df6[['qtde_returns']])
# df6['avg_ticket'] = mm.fit_transform(df6[['avg_ticket']])


In [ ]:
# df6.head()

# Feature selection

In [ ]:
# df7 = df6.copy()

In [ ]:
#cols_select = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_products', 'freq', 'qtde_returns']

In [ ]:
#df7 = df7[cols_select]

# Fine tunning

In [ ]:
#X = df7.drop(columns='customer_id')
X = df_tree

In [ ]:
X.head()

In [ ]:
clusters = np.arange(2,31,1)

## KNN

In [ ]:
sil_k = []
for k in clusters:
    # model definition
    k_model = c.KMeans(n_clusters=k)

    #model training
    k_model.fit(X)

    #model predict
    labels = k_model.predict(X)

    #model performace
    sil_k.append(m.silhouette_score(X,labels,metric='euclidean'))

## GMM

In [ ]:
gmm_k = []
for k in clusters:
    # model definition
    gmm_model = mx.GaussianMixture(n_components=k)

    #model training
    gmm_model.fit(X)

    #model predict
    labels = gmm_model.predict(X)

    #model performace
    gmm_k.append(m.silhouette_score(X,labels,metric='euclidean'))

## H-clustering

In [ ]:
hr_model = hr.linkage( X, 'ward' )

In [ ]:
hc_k = []
for k in clusters:
    # model definition
    hc_model = hr.linkage(X,'ward')

    #model predict
    labels = hr.fcluster(hc_model,k,criterion='maxclust')

    #model performace
    hc_k.append(m.silhouette_score(X,labels,metric='euclidean'))

## DBSCAN

In [ ]:
eps=0.2
min_samples = 20
#model training e predict
dbscan_model = c.DBSCAN(eps=eps,min_samples=min_samples)

#model training e predict
labels = dbscan_model.fit_predict(X)

db_k_tree = m.silhouette_score(X,labels,metric='euclidean')

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nei = NearestNeighbors(n_neighbors=min_samples).fit(X)
dist, indi = nei.kneighbors(X)

## results tree

In [ ]:
df_results_tree = pd.DataFrame({
    'Kmeans': sil_k,
    'GMM': gmm_k,
    'HC': hc_k,
}).T
df_results_tree.columns = clusters
df_results_tree.style.highlight_max(color='lightgreen',axis=1)

In [ ]:
db_k_tree

In [ ]:
df_tree.to_csv('../src/data/df_tree_embedding.csv',index=False)

# Model training

## Final model

In [ ]:
k = 8

In [ ]:
X = df_tree

In [ ]:
gmm_model = mx.GaussianMixture(n_components=k,random_state=32)

gmm_model.fit(X)

labels = gmm_model.predict(X)

In [ ]:
print('SS value',m.silhouette_score(X,labels,metric='euclidean'))

In [ ]:
# visualizer = SilhouetteVisualizer(kmeans,colors='yellowbrick')
# visualizer.fit(X)
# visualizer.finalize()

### Cluster analysis

In [ ]:
df9 = df5.copy()
df9['clusters'] = labels

### UMAP - t-SNE
### Machine leraning - Manifold
PCA - Matrix, Espaço de distâncias - 9 condições - Colorários
UMAP, T-SNE (2009) - Manifold - Topologia

**UMAP**

In [ ]:
# reducer = umap.UMAP(n_neighbors=100,random_state=42)
# embedding = reducer.fit_transform(X)

In [ ]:
# df_viz['embedding_x'] = embedding[:,0]
# df_viz['embedding_y'] = embedding[:,1]

# sns.scatterplot(x='embedding_x',y='embedding_y',hue='clusters', palette=sns.color_palette('hls',n_colors=len(df_viz['clusters'].unique())),data = df_viz)

In [ ]:
#fig = px.scatter_3d(df9,x='recency_days',y='invoice_no',z='gross_revenue',color='clusters')
#fig.show()

### Cluster profile

In [ ]:
df9.head()

In [ ]:
# number of customers
df_cluster = df9[['customer_id','clusters']].groupby('clusters').count().reset_index()
df_cluster['perc_customer'] = (df_cluster['customer_id']/df_cluster['customer_id'].sum())*100
# Avg gross revenue
df_avg_gross_revenue = df9[['gross_revenue','clusters']].groupby('clusters').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_avg_gross_revenue,how='inner',on='clusters')

# avg recency days
df_avg_recency_days =  df9[['recency_days','clusters']].groupby('clusters').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_avg_recency_days,how='inner',on='clusters')

# avg invoice no
df_avg_invoice_no =  df9[['qtde_products','clusters']].groupby('clusters').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_avg_invoice_no,how='inner',on='clusters')

df_ticket =  df9[['freq','clusters']].groupby('clusters').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_ticket,how='inner',on='clusters')

df_ticket =  df9[['qtde_returns','clusters']].groupby('clusters').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_ticket,how='inner',on='clusters')

In [ ]:
df_cluster.sort_values(by='gross_revenue',ascending=False)

6 - Cluster Insiders

5 - Cluster More products

0 - Cluster Spend Money

3 - Cluster Even More products

7 - Cluster Less days

2 - Cluster Less 1k

1 - Cluster Stop Returns

4 - Cluster More buy

# EDA II

In [ ]:
df10 = df9.copy()
df10['clusters'] = df9['clusters']

In [ ]:
df10.head()

# Deploy

In [ ]:
import sqlite3
from sqlalchemy import create_engine

In [ ]:
df10['customer_id'] = df10['customer_id'].astype(int)
df10['recency_days'] = df10['recency_days'].astype(int)
df10['qtde_products'] = df10['qtde_products'].astype(int)
df10['qtde_returns'] = df10['qtde_returns'].astype(int)

In [ ]:
df10.dtypes

## SQLITE

In [ ]:
#Create table

# query_create_table_insiders = """
#     CREATE TABLE INSIDERS (
#         customer_id      INTEGER,
#         gross_revenue    REAL,
#         recency_days     INTEGER,
#         qtde_products    INTEGER,
#         freq             REAL,
#         qtde_returns     INTEGER,
#         clusters         INTEGER
#     )
# """

# conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_create_table_insiders)
# conn.commit()
# conn.close()

# #insert data
# conn = create_engine('sqlite:///insiders_db.sqlite')
# df10.to_sql('insiders',con=conn,if_exists='append',index=False)


In [ ]:
#consulting database
query = """
    SELECT *
    FROM insiders
"""

df = pd.read_sql_query(query,conn)

In [ ]:
df.head()